In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Module 3.2 - Run LandTrendr over Composites

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/2-LandTrendr.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/3-LandTrendr.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/3-LandTrendr.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This notebook uses Landsat and Sentinel 2 composites from the previous notebook inside the LandTrendr temporal setmentation algorithm

Learn more about LandTrendr:
- [LandTrendr Guide](https://emapr.github.io/LT-GEE/)
- [LandTrendr GEE Publication](https://www.mdpi.com/2072-4292/10/5/691)

### Objective

In this tutorial, you learn how to create and manipulate LandTrendr outputs.

This tutorial uses the following Google Cloud services:

- `Google Earth Engine`

The steps performed include:

- Creating LandTrendr outputs
- Manipulating EE array image objects to get meaningful data out of LandTrendr outputs

# Before you begin

### Set your current URL under `workbench_url`
This gives the Map Viewer a url in which to host the viewer we will be generating. 
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/` (See the image below)

![workspace url](img/workspace-url.png)

### Set a folder to use for all exports under `export_path_root` 
This folder should be an assets folder in an existing GEE project. 
* It will be something like `projects/projectID/assets/someFolder`
* This folder does not have to already exist. If it does not exist, it will be created

In [3]:
workbench_url = 'https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/'
export_path_root  = 'projects/rcr-gee/assets/lcms-training'

print('Done')

Done


# Installation
First, install necessary Python packages. Uncomment the first line to upgrade geeViz if necessary.

In [2]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml

import inspect

ee = getImagesLib.ee
Map = getImagesLib.Map

print('Done')

Initializing GEE
Successfully initialized
geeViz package folder: /opt/conda/lib/python3.10/site-packages/geeViz
Done


## Set up your work environment

Create a folder in your export path where you will export the composites. In addition, create a blank image collection where your composites will live.

Currently, when running within Colab, geeView uses a different project to authenticate through, so you may need to make your asset public to view from within Colab.

In [4]:
# Create folder and a collection and make public

export_composite_collection = f'{export_path_root}/lcms-training_module-2_composites'
export_landTrendr_collection = f'{export_path_root}/lcms-training_module-3_landTrendr'

aml.create_asset(export_landTrendr_collection,asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(export_landTrendr_collection,writers = [],all_users_can_read = True,readers = [])

# aml.batchCopy('projects/rcr-gee/assets/landTrendr-lcms-training-module-2',export_landTrendr_collection,outType = 'imageCollection')
# aml.batchDelete(export_landTrendr_collection)
print('Done')

Asset projects/rcr-gee/assets/lcms-training/lcms-training_module-3_landTrendr already exists
Updating permissions for:  projects/rcr-gee/assets/lcms-training/lcms-training_module-3_landTrendr
Done


In [23]:
Map.clearMap()
Map.proxy_url = workbench_url
# Index to use to demonstrate change detection capabilities of LandTrendr
#Choose band or index
#NBR, NDMI, and NDVI tend to work best
#Other good options are wetness and tcAngleBG
# Must include 'red','nir','swir1' in order to visualize as false color composites in next code block
lcms_actual_lt_collection = ee.ImageCollection('projects/lcms-292214/assets/R8/PR_USVI/Base-Learners/LandTrendr-Collection-1984-2020')
print('All bands LCMS uses for LandTrendr:',lcms_actual_lt_collection.aggregate_histogram('band').keys().getInfo())

# We can use any/all of the bands, but generally bands that use nir and swir are most useful
bandNames = ['red','nir','swir1','swir2','NBR','NDVI','brightness','greenness','wetness']

#LandTrendr Params
# run_params ={
#   'timeSeries': (ImageCollection) Yearly time-series from which to extract breakpoints. The first band is used to find breakpoints, and all subsequent bands are fitted using those breakpoints.
#   'maxSegments':            6,\ (Integer) Maximum number of segments to be fitted on the time series.
#   'spikeThreshold':         0.9,\ (Float, default: 0.9) Threshold for damping the spikes (1.0 means no damping).
#   'vertexCountOvershoot':   3,\(Integer, default: 3) The initial model can overshoot the maxSegments + 1 vertices by this amount. Later, it will be pruned down to maxSegments + 1.
#   'preventOneYearRecovery': False,\(Boolean, default: False): Prevent segments that represent one year recoveries.
#   'recoveryThreshold':      0.25,\(Float, default: 0.25) If a segment has a recovery rate faster than 1/recoveryThreshold (in years), then the segment is disallowed.
#   'pvalThreshold':          0.05,\(Float, default: 0.1) If the p-value of the fitted model exceeds this threshold, then the current model is discarded and another one is fitted using the Levenberg-Marquardt optimizer.
#   'bestModelProportion':    0.75,\(Float, default: 0.75) Allows models with more vertices to be chosen if their p-value is no more than (2 - bestModelProportion) times the p-value of the best model.
#   'minObservationsNeeded':  6\(Integer, default: 6) Min observations needed to perform output fitting.
# };
#Define landtrendr params
run_params = { \
  'maxSegments':            6,\
  'spikeThreshold':         0.9,\
  'vertexCountOvershoot':   3,\
  'preventOneYearRecovery': False,\
  'recoveryThreshold':      0.25,\
  'pvalThreshold':          0.05,\
  'bestModelProportion':    0.75,\
  'minObservationsNeeded':  6\
}



# Bring in composites and pull info from them
composites = ee.ImageCollection(export_composite_collection)

props = composites.first().toDictionary().getInfo()

startYear = props['startYear']
endYear = props['endYear']

startJulian = props['startJulian']
endJulian = props['endJulian']

proj = composites.first().projection().getInfo()

# Pull out the crs
# Depending on if a wkt or epsg format is used, it will be stored under a different key
if 'crs' not in proj.keys():
    crs = proj['wkt']
else:
    crs = proj['crs']
    
transform = proj['transform']
scale = None

studyArea = composites.first().geometry()

# Decompress composites by dividing by 10000 for optical bands and add indices
composites = composites.select(['blue','green','red','nir','swir1','swir2']).map(lambda img: img.divide(10000).float().copyProperties(img,['system:time_start']))
composites = composites.map(getImagesLib.simpleAddIndices)\
                      .map(getImagesLib.getTasseledCap)\
                      .map(getImagesLib.simpleAddTCAngles)


print('Done')

All bands LCMS uses for LandTrendr: ['NBR', 'NDMI', 'NDSI', 'NDVI', 'blue', 'brightness', 'green', 'greenness', 'nir', 'red', 'swir1', 'swir2', 'tcAngleBG', 'wetness']
Done


In [24]:
# First let's explore the composites and some attributes that can help understand how well the composites turned out
Map.clearMap()
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse,'Composites')

Map.turnOnInspector()
Map.view()

Adding layer: Composites
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Starting local web server at: http://localhost:8001/geeView/
HTTP server command: "/opt/conda/bin/python" -m http.server  8001
Done
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


127.0.0.1 - - [08/Aug/2023 19:06:11] "GET /geeView/?accessToken=None HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 19:06:12] "GET /geeView/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 19:06:12] "GET /geeView/js/gena-gee-palettes.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 19:06:12] "GET /geeView/js/load.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 19:06:12] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 19:06:12] "GET /geeView/js/lcms-viewer.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 19:06:12] "GET /geeView/images/layer_icon.png HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 19:06:12] "GET /geeView/images/logos_usda-fs.svg HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 19:06:12] "GET /geeView/images/menu-hamburger_ffffff.svg HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 19:06:12] "GET /geeView/images/usfslogo.png HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 19:06:12] "GET /geeView/images/usdalogo.png HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 19:

In [6]:
# Now let's run LandTrendr over NBR
# NBR is the Normmalized Burn Ratio
# It is sensitive to the presence/absence of moisture
# Double click on the output on the map to query it. Notice this output is not immediately useful for change detection or smoothing out a time series of composites
Map.clearMap()

test_band = 'NBR'
run_params['timeSeries'] = composites.select([test_band])
raw_LT = ee.Algorithms.TemporalSegmentation.LandTrendr(**run_params)

Map.addLayer(raw_LT,{},'LT Raw {}'.format(test_band),True)

Map.turnOnInspector()
Map.view()

Adding layer: LT Raw NBR
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


In [15]:
Map.clearMap()
# We will now see how LandTrendr can be used on its own for change detection

# First, choose a change threshold
# Any changes more negative than this value will be flagged as loss
change_threshold = -0.15

# First, we will select the LandTrendr image array output band
lt_array = raw_LT.select(['LandTrendr'])

# This output has 2 dimensions per pixel
# The rows correspond to 0-years, 1-raw input values, 2-LandTrendr fitted output values, and 3-vertex/non vertex

# First, slice the vertices  and use them as a mask to mask out non vertex values
vertices = lt_array.arraySlice(0,3,4)
lt_array = lt_array.arrayMask(vertices)

# In order to perform change detection, we'll need to get the difference between fitted vertex values
# We do this by slicing the first to the next to the last and then the second to the last and subtracting them
left = lt_array.arraySlice(1,0,-1)
right = lt_array.arraySlice(1,1,None)
diff  = left.subtract(right)

# We then slice the right-hand years an dthe fitted vertex values difference and combine them
years = right.arraySlice(0,0,1)
mag = diff.arraySlice(0,2,3)
forSorting = years.arrayCat(mag,0)

# We can then sort using various rows depending on whether we want the highest magnitude change, most recent, etc
# In this example, the sort row will be the magnitude, thus giving us the highest severity change
sorted = forSorting.arraySort(forSorting.arraySlice(0,1,2))

# Convert the sorted array image into a 2-band image
highest_mag_change = sorted.arraySlice(1,0,1).arrayProject([0]).arrayFlatten([['yr','mag']])

# Mask out any change 
highest_mag_change = highest_mag_change.updateMask(highest_mag_change.select(['mag']).lte(change_threshold))

# Pull the los magnitude palette and flip the color order
lossMagPalette = changeDetectionLib.lossMagPalette.split(',')
lossMagPalette.reverse()

# Set up map
Map.addLayer(forSorting,{},'Array for Sorting',False)
Map.addLayer(sorted,{},'Sorted Array',False)
Map.addLayer(highest_mag_change.select(['mag']),{'min':-0.8,'max':-0.15,'palette':lossMagPalette},'Loss Magnitude')
Map.addLayer(highest_mag_change.select(['yr']),{'min':startYear,'max':endYear,'palette':changeDetectionLib.lossYearPalette},'Loss Year')
Map.turnOnInspector()
Map.view()

Adding layer: Array for Sorting
Adding layer: Sorted Array
Adding layer: Loss Magnitude
Adding layer: Loss Year
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


In [16]:
# We will run LandTrendr for each band
#Clear the map in case it has been populated with layers/commands earlier
Map.clearMap()

#Run LANDTRENDR
for bandName in bandNames:

  # Select the band and run LandTrendr
  run_params['timeSeries'] = composites.select([bandName])
  rawLT = ee.Algorithms.TemporalSegmentation.LandTrendr(**run_params)

  Map.addLayer(rawLT,{},'LT Raw {}'.format(bandName),False)

  # Notice the raw LandTrendr output is in GEE's image array format
  # We'll need to manipulate the raw output a bit to save on storage space
  # Mask out non vertex values to use less storage space
  ltArray = rawLT.select(['LandTrendr'])
  rmse = rawLT.select(['rmse'])
  vertices = ltArray.arraySlice(0,3,4)
  ltArray = ltArray.arrayMask(vertices)

  # Mask out all but the year and vertex fited values (get rid of the raw and vertex rows)
  ltArray = ltArray.arrayMask(ee.Image(ee.Array([[1],[0],[1],[0]])))
  rawLTForExport=ltArray.addBands(rmse)
  Map.addLayer(rawLTForExport,{},'LT Vertex Values Only {}'.format(bandName),False)

  # Show how the compressed vertex-only values can be decompressed later
  decompressedC = changeDetectionLib.simpleLTFit(ltArray,startYear,endYear,bandName,True,run_params['maxSegments'])
  Map.addLayer(decompressedC,{'bands':'{}_LT_fitted'.format(bandName),'min':0.2,'max':0.8},'Decompressed LT Output {}'.format(bandName),False)

  # Join the raw and fited values
  fitted = decompressedC.select(['{}_LT_fitted'.format(bandName)])
  ltJoined = getImagesLib.joinCollections(composites.select([bandName]),fitted)
  Map.addLayer(ltJoined,{'bands':'{}_LT_fitted'.format(bandName),'min':0.2,'max':1,'palette':'D80,080'},'Raw and LT Fitted {}'.format(bandName),True)

  # Export LT array image
  # Set some properties that will be uses later
  rawLTForExport = rawLTForExport.set({'startYear':startYear,
                                          'endYear':endYear,
                                          'startJulian':startJulian,
                                          'endJulian':endJulian,
                                          'band':bandName})
  rawLTForExport =rawLTForExport.set(run_params)
  exportName = 'LT_Raw_{}_yrs{}-{}_jds{}-{}'.format(bandName,startYear,endYear,startJulian,endJulian)
  exportPath = export_landTrendr_collection + '/'+ exportName
  # Export output
  getImagesLib.exportToAssetWrapper(rawLTForExport,exportName,exportPath,{'.default':'sample'},studyArea,scale,crs,transform,overwrite=False)

Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

Adding layer: LT Raw red
Adding layer: LT Vertex Values Only red
Adding layer: Decompressed LT Output red
Adding layer: Raw and LT Fitted red
pyramiding object: {'.default': 'sample'}
LT_Raw_red_yrs1984-2022_jds152-151 currently exists or is being exported and overwrite = False. Export not started. Set overwite = True if you would like to overwite any existing asset or asset exporting task
Adding layer: LT Raw nir
Adding layer: LT Vertex Values Only nir
Adding layer: Decompressed LT Output nir
Adding layer: Raw and LT Fitted nir
pyramiding object: {'.default': 'sample'}
LT_Raw_nir_yrs1984-2022_jds152-151 currently exists or is being exported and overwrite = False. Export not started. Set overwite = True if you would like to overwite any existing asset or asset exporting task
Adding layer: LT Raw swir1
Adding layer: LT Vertex Values Only swir1
Adding layer: Decompressed LT Output swir1
Adding layer: Raw and LT Fitted swir1
pyramiding object: {'.default': 'sample'}
LT_Raw_swir1_yrs1984-2

In [25]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
# If you'd like to track the tasks, use this:
# tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(export_landTrendr_collection, type = 'imageCollection')

print('done')

done


In [21]:
# While we can use the LandTrendr output for change detection, LCMS uses it as inputs to change detection, land cover, and land use models
# This will show how the raw LandTrendr array image asset is converted into a time series 
# of annual fitted, segment duration, segment magnitude of change, and slope values
Map.clearMap()

lt_asset = ee.ImageCollection(export_landTrendr_collection)
# Convert stacked outputs into collection of fitted, magnitude, slope, duration, etc values for each year
# While the fitted LandTrendr value is generally of most importance to our models, 
# LandTrendr segment duration, slope, and magnitude of change can also help our models
lt_fit = changeDetectionLib.batchSimpleLTFit(lt_asset,startYear,endYear,None,bandPropertyName='band',arrayMode=True)

# Vizualize image collection that is used as predictors in LCMS models
# When you double-click the output on the map, notice the different values available to the models
Map.addLayer(lt_fit,{'bands':'swir2_LT_fitted,nir_LT_fitted,red_LT_fitted','min':0.15,'max':0.6},'LandTrendr All Predictors Time Series')


Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

Adding layer: LandTrendr All Predictors Time Series
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


In [22]:
Map.clearMap()

# The best way of understanding how LandTrendr contributes to reducing noise in the original composite time series
# is to visualize them side-by-side
# This example takes the fitted values from LandTrendr and shows them along with the original
# Notice many holes are now filled in by LandTrendr
# In general the amount of noise is reduced. There is a risk however of 
# fitting too much and omitting changes such as those seen in 2017 for Hurricane Maria

# Visualize fitted landTrendr composites
fitted_bns = lt_fit.select(['.*_fitted']).first().bandNames()
out_bns = fitted_bns.map(lambda bn: ee.String(bn).split('_').get(0))

# Give same names as composites
lt_synth = lt_fit.select(fitted_bns,out_bns)

# Visualize raw and LandTrendr fitted composites
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse,'Raw Composite Timelapse')

Map.addTimeLapse(lt_synth,getImagesLib.vizParamsFalse,'Fitted LandTrendr Composite Timelapse')


# Join the raw and fited values
ltJoined = getImagesLib.joinCollections(composites.select(bandNames),lt_fit.select(['.*_fitted']))

Map.addLayer(ltJoined,{'min':0.2,'max':1},'Raw and LT Fitted',True)


Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

Adding layer: Raw Composite Timelapse
Adding layer: Fitted LandTrendr Composite Timelapse
['red', 'nir', 'swir1', 'swir2', 'NBR', 'NDVI', 'brightness', 'greenness', 'wetness', 'NBR_LT_fitted', 'NDVI_LT_fitted', 'brightness_LT_fitted', 'greenness_LT_fitted', 'nir_LT_fitted', 'red_LT_fitted', 'swir1_LT_fitted', 'swir2_LT_fitted', 'wetness_LT_fitted']
Adding layer: Raw and LT Fitted
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None
